In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('/home/yoxara/smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.base.model import Model
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V

/tmp/ipykernel_165803/1585501929.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp


3.0.0-beta


In [2]:
BSMlist = getParticlesFromSLHA('TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[y1]


In [3]:
slhaFolder = 'TRV1qq_slha'
resultsFolder = 'TRV1qq_results'
File = 'parameters.ini'
!chmod -R +rwx {slhaFolder}

In [4]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadBinaryFile() in 549: loading binary db file /home/yoxara/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 556: Loaded database from /home/yoxara/smodels-database/db3.pcl in 2.2 secs.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 205 files with a single process. Messages will be redirected to smodels.log


In [5]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

In [6]:
print(len(data))

205


In [7]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_167_MZp_1700_gB_1.349141E+00.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",7.737691e+01,0.000000e+00,2.321800e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_167_MZp_1700_gB_1.349141E+00.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,1.548031e+02,1.219720e+02,NaN,[TRV1qq],"[(y1, 1700.4)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,1.269169e+00,NaN,"[(y1, 4.1046)]"
1,run_48_MZp_649_gB_1.154425E+00.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",7.193188e+03,0.000000e+00,2.182000e+04,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_48_MZp_649_gB_1.154425E+00.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,1.462681e+04,1.012840e+04,NaN,[TRV1qq],"[(y1, 649.2)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,1.444138e+00,NaN,"[(y1, 1.1412)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,run_14_mzp_1430_gB_0.6277443.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.727314e+01,0.000000e+00,1.418800e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_14_mzp_1430_gB_0.6277443.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,9.460689e+01,1.374220e+02,NaN,[TRV1qq],"[(y1, 1430.0)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,6.884406e-01,NaN,"[(y1, 7.4722)]"
204,run_128_MZp_1250_gB_1.349141E+00.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.577295e+02,0.000000e+00,1.374200e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_128_MZp_1250_gB_1.349141E+00.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,9.164705e+02,1.336140e+02,NaN,[TRV1qq],"[(y1, 1250.1)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,6.859090e+00,NaN,"[(y1, 3.0167)]"


In [8]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 5000001])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [9]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [10]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [11]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 205


In [12]:
#Save DataFrame to pickle file:
dataDF.to_pickle('TRV1qq_data.pcl')
print(len(dataDF))
dataDF

205


,filename,mass.5000001,width.5000001,extpar.1,xsec8TeV(fb).5000001,"BRs.y1.b,b","BRs.y1.c,c","BRs.y1.q,q","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_167_MZp_1700_gB_1.349141E+00.slha,1.700433e+03,4.104564e+00,1.349141e+00,2.321800e+02,1.666844e-01,1.666844e-01,1.666844e-01,1.665782e-01,0.000000e+00,2.321800e+02,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",7.737691e+01,0.000000e+00,2.321800e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_167_MZp_1700_gB_1.349141E+00.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,1.548031e+02,1.219720e+02,NaN,[TRV1qq],"[(y1, 1700.4)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,1.269169e+00,NaN,"[(y1, 4.1046)]"
1,run_48_MZp_649_gB_1.154425E+00.slha,6.491725e+02,1.141154e+00,1.154425e+00,2.182000e+04,1.675849e-01,1.675849e-01,1.675849e-01,1.620754e-01,0.000000e+00,2.182000e+04,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",7.193188e+03,0.000000e+00,2.182000e+04,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_48_MZp_649_gB_1.154425E+00.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,1.462681e+04,1.012840e+04,NaN,[TRV1qq],"[(y1, 649.2)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,1.444138e+00,NaN,"[(y1, 1.1412)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,run_14_mzp_1430_gB_0.6277443.slha,1.430000e+03,7.472187e+00,6.277443e-01,1.418800e+02,1.667023e-01,1.667023e-01,1.667023e-01,1.664887e-01,0.000000e+00,1.418800e+02,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.727314e+01,0.000000e+00,1.418800e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1qq_slha/run_14_mzp_1430_gB_0.6277443.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,9.460689e+01,1.374220e+02,NaN,[TRV1qq],"[(y1, 1430.0)]",ATLAS-EXOT-2013-11,NaN,8.000000e+00,2.030000e+01,upperLimit,6.884406e-01,NaN,"[(y1, 7.4722)]"
204,run_128_MZp_1250_gB_1.349141E+00.slha,1.250075e+03,3.016684e+00,1.349141e+00,1.374200e+03,1.667280e-01,1.667280e-01,1.667280e-01,1.663600e-01,0.000000e+00,1.374200e+03,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.577295e+02,0.000000e+00,1.374200e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1qq_slha/run_42_MZp_649_gB_4.599776E-01.slha,1.000000e+00,1.000

In [13]:
dataDF['xsec8TeV(fb).5000001'] *(3 *dataDF['BRs.y1.q,q'] + dataDF['BRs.y1.c,c'])


0     1.548031e+02
1     1.462681e+04
          ...     
203   9.460689e+01
204   9.164705e+02
Length: 205, dtype: float64

In [14]:
dataDF['ExptRes.result0.theory prediction (fb)']

0     1.548031e+02
1     1.462681e+04
          ...     
203   9.460689e+01
204   9.164705e+02
Name: ExptRes.result0.theory prediction (fb), Length: 205, dtype: float64

In [15]:
(dataDF['xsec8TeV(fb).5000001'] *(3 *dataDF['BRs.y1.q,q'] + dataDF['BRs.y1.c,c']))/dataDF['ExptRes.result0.theory prediction (fb)']

0     1.000000e+00
1     1.000000e+00
          ...     
203   1.000000e+00
204   1.000000e+00
Length: 205, dtype: float64

In [16]:
dataDF['ExptRes.result0.r']

0     1.269169e+00
1     1.444138e+00
          ...     
203   6.884406e-01
204   6.859090e+00
Name: ExptRes.result0.r, Length: 205, dtype: float64